# Segmenting and Clustering Neighborhoods in Toronto

**Author:** Victor Soeiro Araujo Pereira

**Description:** This notebook is a assignment for the week 3 of the final course on Data Science Professional Certificate. We gonna use some knowledge we gained of Python during the ten courses. We will segmenting and clustering some info of foursquare in the city of Toronto in Canada. 

## Setup

Importing the modules we gonna use.

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from bs4 import BeautifulSoup

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Importing data

Let's request the HTML from the wikipedia page.

In [3]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

if r.status_code == 200:
    print('Request was successful.')

Request was successful.


Now, let's scrape the data from the HTML using BeautifulSoup.

In [4]:
b = BeautifulSoup(r.text, 'html.parser')

data = []
for row in b.find('tbody').find_all('td'):
    if row.span.text == 'Not assigned':
        continue
    
    postal_code = row.find('b').text
    borough = row.find('span').text.split('(')[0]
    neighborhood = row.find('span').text.split('(')[1].replace(')', '').replace(' / ', ', ')
    
    data.append([postal_code, borough, neighborhood])
    
df = pd.DataFrame(data, columns=['Postal Code', 'Borough', 'Neighborhood'])
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Clean Data

We need to clean some data, because some Borough is namely wrong. Replacing it's names.

In [5]:
df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',
                       'EtobicokeNorthwest': 'Etobicoke Northwest',
                       'East YorkEast Toronto': 'East York / East Toronto',
                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
                       'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'}, inplace=True)

If some Neighborhood is not assigned, it will be equal to the borough.

In [6]:
df[df['Neighborhood'] == 'Not assigned'].value_counts()

Series([], dtype: int64)

## Shape

We know it has three columns, but how many rows?

In [7]:
df.shape

(103, 3)

And how many Boroughs?

In [8]:
df['Borough'].nunique()

15

## Latitude and Longitude

The assignment gave us an URL containing the latitude and longitude for every postal code. Pandas module has an useful function to read csv files, even if they need to be downloaded given an URL.

In [9]:
geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two DataFrames on the 'Postal Code' column.

In [10]:
df2 = pd.merge(df, geo, on='Postal Code')
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Segmenting and Clustering

### Exploring

Let's first explore and create the Toronto map. 

In [11]:
toronto_lat = 43.7134817
toronto_lng = -79.3839347

toronto_map = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=10)
for idx, row in df2.iterrows():
    label = folium.Popup(f'{row["Neighborhood"]}, {row["Borough"]}', parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=4,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False
    ).add_to(toronto_map)
    
toronto_map

### Foursquare API

To use the Foursquare API we need the Client ID and Client Secret, registering a free account on Foursquare Developers. For security purposes I saved my credentials on a json file. Let's load the credential information

In [12]:
import json

with open('credentials.json', 'r') as file:
    credentials = json.load(file)

Let's create a function to get the nearby venues given a radius, latitude, longitude and credentials. 

In [13]:
def get_venues(radius, latitude, longitude, credentials, limit=100):    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        credentials['id'], credentials['secret'], credentials['version'], latitude, longitude, radius, limit)
    
    r = requests.get(url)
    if r.status_code != 200:
        print('Invalid request.')
    
    return r.json()

For tests purpose we will use one of the Borough.

In [14]:
data = get_venues(500, df2.loc[20, 'Latitude'], df2.loc[20, 'Longitude'], credentials)

How many results were returned?

In [15]:
print('{} venues were returned by Foursquare API.'.format(data['response']['totalResults']))

60 venues were returned by Foursquare API.


Cool! From the previous lab we know that the venues are on items key. Before to proceed, we will borrow the function that get's the category name of the venue to apply to the dataframe.

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now, we can clean the json file and build our DataFrame.

In [17]:
venues = data['response']['groups'][0]['items']

venues_df = pd.json_normalize(venues)
venues_df = venues_df[['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]
venues_df['venue.categories'] = venues_df.apply(get_category_type, axis=1)
venues_df.columns = ['Name', 'Categories', 'Latitude', 'Longitude']
venues_df.head()

,Name,Categories,Latitude,Longitude
0,LCBO,Liquor Store,43.642944,-79.372440
1,The Keg Steakhouse + Bar - Esplanade,Restaurant,43.646712,-79.374768
2,Fresh On Front,Vegetarian / Vegan Restaurant,43.647815,-79.374453
3,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323
4,Goose Island Brewhouse,Beer Bar,43.647329,-79.373541


Let's make the same thing for every Borough.

In [18]:
venues_list = list()
for idx, row in df2.iterrows():
    print('Getting data from', row['Neighborhood'])
    results = get_venues(500, row['Latitude'], row['Longitude'], credentials)['response']['groups'][0]['items']
    
    venues_list.extend([(
        row['Neighborhood'],
        row['Latitude'],
        row['Longitude'],
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results
    ])
    
    print('Done.')
    
venues_df = pd.DataFrame(venues_list, columns=[
    'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
    'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'
])

Getting data from Parkwoods
Done.
Getting data from Victoria Village
Done.
Getting data from Regent Park, Harbourfront
Done.
Getting data from Lawrence Manor, Lawrence Heights
Done.
Getting data from Ontario Provincial Government
Done.
Getting data from Islington Avenue
Done.
Getting data from Malvern, Rouge
Done.
Getting data from Don MillsNorth
Done.
Getting data from Parkview Hill, Woodbine Gardens
Done.
Getting data from Garden District, Ryerson
Done.
Getting data from Glencairn
Done.
Getting data from West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Done.
Getting data from Rouge Hill, Port Union, Highland Creek
Done.
Getting data from Don MillsSouth
Done.
Getting data from Woodbine Heights
Done.
Getting data from St. James Town
Done.
Getting data from Humewood-Cedarvale
Done.
Getting data from Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Done.
Getting data from Guildwood, Morningside, West Hill
Done.
Getting data from The Beaches
Done.
Gettin

### Check the resulting DataFrame

In [19]:
venues_df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


And now the shape.

In [20]:
venues_df.shape

(2111, 7)

How many venues of each Neighborhood?

In [22]:
venues_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,60,60,60,60,60,60
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15


And how many categories?

In [23]:
print('There are {} unique categories.'.format(venues_df['Venue Category'].nunique()))

There are 269 unique categories.


### Analyzing Categories

Let's transform each unique category in a column using the get_dummies function of the module pandas.

In [24]:
toronto_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix='', prefix_sep='')
toronto_onehot['Neighborhood'] = venues_df['Neighborhood']
toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]
toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
toronto_onehot.shape

(2111, 269)

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [27]:
toronto_grouped.shape

(98, 269)

### Top Categories

In [28]:
def return_top_venues(row, top_venues):
    categories = row.iloc[1:]
    categories_sorted = categories.sort_values(ascending=False)
    return categories_sorted.index.values[0:top_venues]

In [29]:
top_venues = 10

columns = [f'Top {i}' for i in range(1, top_venues+1)]
columns.insert(0, 'Neighborhood')

neighborhoods_venues = pd.DataFrame(columns=columns)
neighborhoods_venues['Neighborhood'] = toronto_grouped['Neighborhood']

for i in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues.iloc[i, 1:] = return_top_venues(toronto_grouped.iloc[i, :], top_venues)

In [30]:
neighborhoods_venues.head()

,Neighborhood,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Clothing Store,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Coffee Shop,Pub,Yoga Studio,Middle Eastern Restaurant,Modern European Restaurant,Mobile Phone Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Sushi Restaurant,Frozen Yogurt Shop,Restaurant,Supermarket,Fried Chicken Joint,Gas Station,Diner
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Pizza Place,Toy / Game Store,Sushi Restaurant,Pub,Café,Thai Restaurant


### Clustering

In [31]:
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=4)
kmeans.fit(toronto_grouped.drop('Neighborhood', 1))

kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 0])

In [32]:
neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_df = df2
toronto_df = toronto_df.join(neighborhoods_venues.set_index('Neighborhood'), on='Neighborhood')

In [33]:
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Park,Yoga Studio,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Pizza Place,Financial or Legal Service,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Shoe Store,Beer Store,French Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Furniture / Home Store,Clothing Store,Miscellaneous Shop,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Café,Beer Bar,Fried Chicken Joint,Creperie,Smoothie Shop,Mexican Restaurant,Burrito Place


In [34]:
toronto_df['Cluster Labels'] = toronto_df['Cluster Labels'].fillna(0.0)

In [35]:
map_clusters = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for idx, row in toronto_df.iterrows():
    label = folium.Popup(str(row['Neighborhood']) + ' Cluster ' + str(row['Cluster Labels']), parse_html=True)
        
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color=rainbow[int(row['Cluster Labels'])-1],
        fill=True,
        fill_color=rainbow[int(row['Cluster Labels'])-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [36]:
toronto_df.loc[toronto_df['Cluster Labels'] == 0, toronto_df.columns[[1] + list(range(5, toronto_df.shape[1]))]]

,Borough,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
0,North York,0.0,Food & Drink Shop,Park,Yoga Studio,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store
5,Etobicoke,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,York,0.0,Field,Hockey Arena,Trail,Park,Historic Site,Moroccan Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant
21,York,0.0,Park,Pool,Women's Store,Health & Beauty Service,Health Food Store,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
35,East York / East Toronto,0.0,Park,Convenience Store,Intersection,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
40,North York,0.0,Park,Airport,Yoga Studio,Metro Station,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
45,North York,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,North York,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,Central Toronto,0.0,Park,Bus Line,Swim School,Yoga Studio,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
64,York,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 2

In [37]:
toronto_df.loc[toronto_df['Cluster Labels'] == 1, toronto_df.columns[[1] + list(range(5, toronto_df.shape[1]))]]

,Borough,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
10,North York,1.0,Pizza Place,Park,Japanese Restaurant,Bakery,Yoga Studio,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
11,Etobicoke,1.0,Bakery,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Lounge
44,Scarborough,1.0,Bakery,Soccer Field,Park,Intersection,Bus Station,Metro Station,Ice Cream Shop,Bus Line,Mobile Phone Shop,Modern European Restaurant
49,North York,1.0,Basketball Court,Park,Construction & Landscaping,Bakery,Yoga Studio,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant


#### Cluster 3

In [38]:
toronto_df.loc[toronto_df['Cluster Labels'] == 2, toronto_df.columns[[1] + list(range(5, toronto_df.shape[1]))]]

,Borough,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
6,Scarborough,2.0,Fast Food Restaurant,Moroccan Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


#### Cluster 4

In [39]:
toronto_df.loc[toronto_df['Cluster Labels'] == 3, toronto_df.columns[[1] + list(range(5, toronto_df.shape[1]))]]

,Borough,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
57,North York,3.0,Baseball Field,Yoga Studio,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Lounge
101,Etobicoke,3.0,Baseball Field,Yoga Studio,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Lounge


#### Cluster 5

In [40]:
toronto_df.loc[toronto_df['Cluster Labels'] == 4, toronto_df.columns[[1] + list(range(5, toronto_df.shape[1]))]]

,Borough,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
1,North York,4.0,Pizza Place,Financial or Legal Service,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
2,Downtown Toronto,4.0,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Shoe Store,Beer Store,French Restaurant
3,North York,4.0,Furniture / Home Store,Clothing Store,Miscellaneous Shop,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
4,Queen's Park,4.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Café,Beer Bar,Fried Chicken Joint,Creperie,Smoothie Shop,Mexican Restaurant,Burrito Place
7,North York,4.0,Café,Dessert Shop,Gym,Athletics & Sports,Caribbean Restaurant,Japanese Restaurant,Modern European Restaurant,Motel,Moroccan Restaurant,Monument / Landmark
8,East York,4.0,Pizza Place,Athletics & Sports,Café,Flea Market,Bank,Intersection,Gastropub,Pharmacy,Gym / Fitness Center,Movie Theater
9,Downtown Toronto,4.0,Coffee Shop,Clothing Store,Hotel,Italian Restaurant,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Diner
12,Scarborough,4.0,Home Service,Bar,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Yoga Studio,Moroccan Restaurant
13,North York,4.0,Coffee Shop,Restaurant,Gym,Bike Shop,Beer Store,Asian Restaurant,Discount Store,Italian Restaurant,Art Gallery,Dim Sum Restaurant
14,East York,4.0,Intersection,Park,Athletics & Sports,Beer Store,Bus Stop,Skating Rink,Curling Ice,Miscellaneous Shop,Molecular Gastronomy Restaurant,Modern European Restaurant
